In [1]:
import json
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import psycopg2
import math
from tqdm.auto import tqdm

In [ ]:
def y_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    return data['translate']
y_translate("привет уеба", "ru", "fr")

In [3]:
def check_unhandled_lookup(directory):
    total_files = 0
    unhandled_lookup = 0
    unhandled_words = []
    for file in tqdm(os.listdir(directory)):
        if "DS_Store" not in file:
            try:
                f = open(os.path.join(directory,file), "r", encoding='utf-8')
            except:
                f = open(os.path.join(directory,file), "r")
            try:
                data = json.load(f, encoding = 'utf-8')
            except:
                print(file)
                data = json.load(f)
            #print(len(data['def']), data['def'])
            if len(data['def']) == 0:
                unhandled_lookup += 1
                word = file.split(".")[0]
                unhandled_words.append(word)
            total_files +=1
    return total_files, unhandled_lookup, unhandled_words
total_json, unhandled_json, unhandled_words = check_unhandled_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru")


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/Users/nigula/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-bf551a7a4abf>", line 23, in <module>
    total_json, unhandled_json, unhandled_words = check_unhandled_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru")
  File "<ipython-input-3-bf551a7a4abf>", line 8, in check_unhandled_lookup
    f = open(os.path.join(directory,file), "r", encoding='utf-8')
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nigula/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Trace

KeyboardInterrupt: 

In [17]:
def parse_from_yandex_lookup(directory, print_output = False):
    word = []
    local_word = []
    pos_list = []
    examples = []
    examples_local = []
    words_from_not_handled_count = 0 
    empty = 0
    for file in tqdm(os.listdir(directory)):
        if print_output == True: print(file)
        current_word = file.split(".")[0]
        
        if "DS_Store" not in file:
            try:
                f = open(os.path.join(directory,file), "r", encoding='utf-8')
            except:
                f = open(os.path.join(directory,file), "r")
            try:
                data = json.load(f, encoding = 'utf-8')
            except:
                data = json.load(f)
            if "def" in data:
                for definition in data['def']:
                    word_current =  definition['text']
                    if 'pos' in definition:
                        global_pos = definition['pos']
                    else:
                        global_pos = 'no_pos_available'
                    for translation in definition['tr']: 
                        if 'pos' in translation:
                            pos = translation['pos']
                        else:
                            pos = global_pos
                        pos_list.append(pos)
                        word.append(word_current)
                        local_word.append(translation['text'])
                        ex_en = []
                        ex_rus = []
                        if 'ex' in translation:
                            for exmpl in translation['ex']:
                                #print(exmpl)
                                ex_en.append(exmpl['text'])
                                ex_rus.append(exmpl['tr'][0]['text'])
                        examples.append(ex_en)
                        examples_local.append(ex_rus)
                if len(data['def']) == 0:
                    empty += 1
            words_from_not_handled_count += 1
    print("empty json", empty)
    print("non_handled_basic_language__words_added_to_dataframe", words_from_not_handled_count)
    data = pd.DataFrame(list(zip(word,local_word,pos_list, examples, examples_local)),columns =['word', 'local_word', 'pos', 'examples','local_examples'])
    return  data


In [ ]:
os.list

In [33]:
yandex_lookup_fr_en = parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en")

empty json 26409
non_handled_basic_language__words_added_to_dataframe 45262


In [34]:
yandex_lookup_fr_en_2 = parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2")

empty json 4904
non_handled_basic_language__words_added_to_dataframe 9245


In [43]:
df_fr_en = pd.concat([yandex_lookup_fr_en, yandex_lookup_fr_en_2])
del df_fr_en['examples']
del df_fr_en['local_examples']

df_fr_en.head()

,word,local_word,pos
0,Sagittaire,archer,noun
1,Sagittaire,Sagittarian,noun
2,Zorn,Zorn,noun
3,Guantanamo,Guantanamo,noun
4,Guantánamo,Guantanamo,noun


In [38]:
len(set(yandex_lookup_fr_en_2['word'])) + 4904

9272

In [39]:
df_fr_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32250 entries, 0 to 5961
Data columns (total 5 columns):
word              32250 non-null object
local_word        32250 non-null object
pos               32250 non-null object
examples          32250 non-null object
local_examples    32250 non-null object
dtypes: object(5)
memory usage: 1.5+ MB


In [45]:
df_fr_en = df_fr_en.drop_duplicates()

In [48]:
len(set(df_fr_en['word'])) 

21750

In [46]:
df_fr_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29946 entries, 0 to 5961
Data columns (total 3 columns):
word          29946 non-null object
local_word    29946 non-null object
pos           29946 non-null object
dtypes: object(3)
memory usage: 935.8+ KB


In [47]:
df_fr_en.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_en.csv")

In [ ]:
Боевой ru_fr (на момент начала парсинга все слов собраны)

In [7]:
yandex_lookup_ru_fr = parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_ru_fr")


non_handled_basic_language__words_added_to_dataframe 68545


In [8]:
yandex_lookup_ru_fr.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_ru_fr.csv")

In [ ]:
Боевой en_ru

In [75]:
not_handled_in_triplet_eng_word_csv = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos/not_handled_in_triplet_eng_word.csv")
not_handled_in_triplet_eng_word_set= set(not_handled_in_triplet_eng_word_csv['words_no_handled'])
len(not_handled_in_triplet_eng_word_set)


46014

In [78]:
yandex_lookup_en_ru = parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru", 
                                              not_handled_words=not_handled_in_triplet_eng_word_set)


basic_words_added_to_dataframe 46012


In [76]:
len(yandex_lookup_en_ru)

91843

In [81]:
yandex_lookup_en_ru.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_non_handled_friday.csv")



In [4]:
yandex_lookup_fr_en= parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_fr_en")

In [5]:
yandex_lookup_fr_en.head()

,word,local_word,pos,examples,local_examples
0,tomber,fall,verb,[],[]
1,tomber,fell,noun,[],[]
2,tomber,fall down,verb,[],[]
3,pénétrer,penetrate,verb,[],[]
4,complet,full,noun,[],[]


In [4]:
yandex_lookup_fr_ru= parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_ru")

In [5]:
yandex_lookup_fr_ru.head()

,word,local_word,pos,examples,local_examples
0,tomber,упасть,verb,"[branches tombées, tomber à terre, tomber vict...","[упавшие ветви, падать на землю, пасть жертвам..."
1,tomber,попасть,verb,[tomber en disgrâce],[попадать в немилость]
2,tomber,опускать,verb,[],[]
3,tomber,опадать,verb,[],[]
4,tomber,положить,verb,[],[]


In [6]:
yandex_lookup_fr_ru.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_ru.csv")

In [6]:
yandex_lookup_fr_en.to_csv("yandex_lookup_fr_en.csv")

In [17]:
yandex_lookup_en_ru_new.head()

,word,local_word,pos,examples,local_examples
0,youngling,детеныш,noun,[],[]
1,youngling,юнец,noun,[],[]
2,asperity,лишения,noun,[],[]
3,asperity,шероховатость,noun,[],[]
4,asperity,резкость,noun,[],[]


In [ ]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")


In [ ]:
def parse_json(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [ ]:
def parse_from_yandex_lookup_vs_wordforms(directory):
    word = []
    local_word = []
    examples = []
    examples_local = []
    words = pd.read_csv("test_words.csv", header = None)
    for word_i in words[0].items():
        print (word_i[1])
        file = word_i[1] + ".json"
        parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        wordforms = get_word_forms(word_i[1].lower())
        for wf in wordforms:
            print(">>>", wf)
            file = wf + ".json"
            parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data
enru_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_ru/")

In [ ]:
enru_wv.to_csv("yandex_lookup_en_ru_with_wordforms.csv")

In [ ]:
enru_wv.head(30)

In [ ]:
en_fr_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_fr/")

In [ ]:
en_fr_wv.to_csv("yandex_lookup_en_fr_with_wordforms.csv")

In [ ]:
len(word), len(local_word), len(examples), len(examples_local)

In [ ]:
enru_ = parse_from_yandex_lookup("./y_lookup/en_ru/")
enru.head()

In [ ]:
enru

In [ ]:
enfr__whole_list = parse_from_yandex_lookup("./y_lookup/en_fr_whole_list/", print_output=True)
enfr__whole_list.head()

In [ ]:
list(enfr__whole_list['word']).index("nobody")

In [ ]:
enru.to_csv("yandex_lookup_en_ru_lemmas_only.csv")

In [ ]:
enfr__whole_list.to_csv("yandex_lookup_en_fr_lemmas_only_WHOLE_LIST.csv")

In [ ]:
url_y = "http://192.168.122.13:31436/GetTranslate"
h_y = {"apiVersion": "1.0.0", "text": "гайки и болты", "langPair": {"source": "ru", "target": "fr"}}
#h_y = {"apiVersion": "1.0.0", "text": "Landing to the mars", "langPair": "en-ru"}
r = requests.post(url = url_y, json=h_y)
data = r.json()
data

'salut, connard.'

In [ ]:
enru_small = parse_from_yandex_lookup("./y_lookup/en_ru/")
len(enru_small)

In [4]:
enru_big.head()

,word,local_word,examples,local_examples
0,before,до,[before the war],[до войны]
1,before,впереди,[],[]
2,before,пред,[],[]
3,before,накануне,[],[]
4,before,передо,[],[]


In [5]:
enru_big.to_csv("./lookup results/yandex_lookup_en_ru_lemmas_only.csv")

In [ ]:
def naive_translation(en_ru_dataframe, from_lang,to_lang):
    word = []
    local_word = []
    example = []
    local_example = []
    for element_ind in tqdm(range(len(en_ru_dataframe))):
        el_list = list(en_ru_dataframe.iloc[element_ind])
        word.append(el_list[1])
        print(el_list)
        #print(el_list[2])
        
        try:
            tr_local_word = y_translate(el_list[2],from_lang,to_lang)
            print(el_list[2], "ok")
        except:
            tr_local_word = "no_translation"
            print("failed", el_list[1])
        local_word.append(tr_local_word)
        ex = ''
        loc_ex = ''
        try:
            for example_i in el_list[3].split("|"):
                if example_i:
                    ex += example_i + '|'
                    try:
                        ex_tr = y_translate(example_i,"en",to_lang)
                        print(example_i, "ok")
                    except:
                        ex_tr = "no_translation"
                        print("failed", example_i)
                    loc_ex+= ex_tr + '|'
        except:
            print("no examples")
        ex = ex[:-1]
        loc_ex = loc_ex[:-1]
        example.append(ex)
        local_example.append(loc_ex)
        if element_ind % 10 == 0: 
            data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
            data.to_csv("/Users/nigula/LL/adjust_unigr_dict/y_translate/systran_enrufr_naive/"+str(element_ind)+".csv")
        
        #if element_ind > 10: break
    data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
    return data

In [ ]:
df_enru_whole = naive_translation(enru_big,"ru","fr")
df_enru_whole.head()

In [ ]:
df_enru_whole.to_csv("naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")

# SYSTRAN EN RU TO FR NAIVE

In [ ]:
df_systran_en_ru = pd.read_csv("systran_en_ru.csv")

In [ ]:
df_systran_en_ru.head(12)

In [ ]:
df_systran_en_ru_fr = naive_translation(df_systran_en_ru,"ru","fr")

In [ ]:
list(df_systran_en_ru['local_word'])

In [ ]:
df_systran_en_ru_fr['rus_word'] = list(df_systran_en_ru['local_word'])

In [ ]:
df_systran_en_ru_fr = df_systran_en_ru_fr.reindex(columns = ['word','rus_word', 'local_word','examples','local_examples'])

In [ ]:
df_systran_en_ru_fr.head(100)

In [ ]:
df_systran_en_ru_fr.to_csv("enrufr_naive_translation_systran_lookup_whole_list.csv")